In [ ]:
import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp
from collections import namedtuple

In [ ]:
np.random.seed(42)

In [ ]:
with open('../data/Scholl/GUNTHER.IN2', 'r') as f:
    text = f.readlines()

In [ ]:
relation = namedtuple('Relation', ['pre', 'suc'])
task = namedtuple('Task', ['start', 'stop', 'interval'])

In [ ]:
params = {}
text = [x.replace('\n', '') for x in text]
params['task_num'] = int(text[0])

In [ ]:
relations = []
for i in range(1, params['task_num'] + 1):
    params[i] = int(text[i])
for i in range(params['task_num'] + 1, len(text) - 1):
    relations.append(relation(*map(lambda x: int(x), text[i].split(','))))
horizon = sum([t for t in list(params.values())[1:]])

In [ ]:
relations

# Create Variables

In [ ]:
solver = pywraplp.Solver('simple_scheduling_programm', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
solver.Clear()
solver

In [ ]:
tasks = [[]] * params['task_num']
for i in range(params['task_num']):
    start = solver.IntVar(0, horizon, 'start_' + str(i+1))
    duration = params[i + 1]
    end =  start + duration
    tasks[i] = task(start=start, stop=end, interval=duration)

print('Number of variables:', solver.NumVariables())

# Create Objective Function

In [ ]:
solver.Minimize(solver.Sum([task.stop for task in tasks]))
# solver.Minimize(solver.Sum( [tasks[i] for i in range(len(tasks))] ))

# Create Constraints

In [ ]:
for i in range(len(tasks) - 1):
    solver.Add(tasks[i + 1].start >= tasks[i].start + tasks[i].interval)

print('Number of constraints:', solver.NumConstraints())

In [ ]:
solver.Solve()

for t in tasks:
    print('Task t: {:<10} | start: {:<10} | stop: {:<10}'.format(str(t.start).split('_')[-1], str(t.start.solution_value()), str(t.stop.solution_value())))